# 07: Data Leakage

In [1]:
import pandas as pd

In [2]:
# read in the data
data = pd.read_csv('../datasets/AER_credit_card_data.csv', true_values = ['yes'], false_values = ['no'])

# select target
y = data.card

# select predictors
X = data.drop(columns=['card'])

print(f'# of rows: {X.shape[0]}')
display(X.head())

# of rows: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

my_pipeline = Pipeline(steps=[('model', RandomForestClassifier(n_estimators=100))])
cv_scores = cross_val_score(my_pipeline, X, y, cv=5, scoring='accuracy')

print(f'X-val accuracy: {cv_scores.mean()}')

X-val accuracy: 0.9810519645120406


In [7]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


In [8]:
# drop leaks
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(columns=potential_leaks)

#Evaluate model
cv_scores = cross_val_score(my_pipeline, X2, y, cv=5, scoring='accuracy')
print(f'Mean CV: {cv_scores.mean()}')

Mean CV: 0.8286553750432077
